In [1]:
import sys
import os
import json
import tensorflow as tf
import tqdm
from models.model import *
from ltv_utils import *
pd.set_option('display.float_format', '{:.4f}'.format)  # 保留10位小数，可调整
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)




def parse_function(serialized_example):
    feature_description = {
        'deviceid': tf.io.FixedLenFeature([], tf.string),
        'install_date': tf.io.FixedLenFeature([], tf.string),
        'dim_os_name1': tf.io.FixedLenFeature([], tf.string),
        'creative_classify1': tf.io.FixedLenFeature([], tf.string),
        'total_pay_amount1':  tf.io.FixedLenFeature([], tf.float32),
         'channel1': tf.io.FixedLenFeature([], tf.string),
        'b2_sale_amt_bias':  tf.io.FixedLenFeature([], tf.int64),
         'b2_sale_amt_7d': tf.io.FixedLenFeature([], tf.int64),
         'install_time': tf.io.FixedLenFeature([], tf.string),
        'install_order_diff':  tf.io.FixedLenFeature([], tf.int64),
        'all_install_order_7d_diff':  tf.io.FixedLenFeature([], tf.int64),
        'is_a1x_a33':  tf.io.FixedLenFeature([], tf.int64),
        'platform_label':  tf.io.FixedLenFeature([], tf.string),
        'user_dense_price_features': tf.io.FixedLenFeature([len(group_2_features['user_dense_price_features'])], tf.float32),
        'user_dense_duration_features': tf.io.FixedLenFeature([len(group_2_features['user_dense_duration_features'])], tf.float32),
        'user_dense_features': tf.io.FixedLenFeature([len(group_2_features['user_dense_features'])], tf.float32),
        'user_sparse_features': tf.io.FixedLenFeature([len(group_2_features['user_sparse_features'])], tf.float32)
    }
    example = tf.io.parse_single_example(serialized_example, feature_description)
    return example


# load tf records
group_2_features = read_feature_json_config('features/feature_list.json')
file_name = 'data/loca_test_tf.tfrecords'
data_path = file_name

dataset = tf.data.TFRecordDataset(data_path)
dataset = dataset.map(parse_function)

dataset = dataset.prefetch(buffer_size=10000)
dataset = dataset.batch(512)

user_dense_price_features = group_2_features['user_dense_price_features']
user_dense_duration_features = group_2_features['user_dense_duration_features']
user_dense_features = group_2_features['user_dense_features']
user_sparse_features = group_2_features['user_sparse_features']


In [2]:
emb_features = [
'creative_classify','dim_device_manufacture', 'car_add_type_most','show_order_is_2arrival_latest', 'selecttirecount_most', 'show_order_is_2arrival_most','selecttirecount_latest',
 'new_sitename','advsite','car_add_type_latest','platform_level', 'tire_list_click_avg_index','tire_list_click_most_pid_level','tire_order_page_most_pid_level',
]


model = MULTI_HEAD_LTV_MODEL(5, [200,200], [200,128], 'user_dense_features', 'user_dense_price_features', 'user_dense_duration_features',
                            'user_sparse_features',user_sparse_features, emb_features)

In [3]:
for batch in dataset.take(1):
    res =  model(batch)
res 

tf.Tensor(
[2 0 1 0 2 1 0 4 1 2 0 3 0 1 2 2 3 3 3 2 2 1 2 2 0 1 1 3 4 2 0 3 2 1 2 1 3
 3 2 1 2 4 0 3 1 0 0 1 0 3 4 1 2 0 1 0 0 2 4 3 2 4 1 4 3 0 3 4 1 2 3 2 2 2
 3 2 1 4 3 1 3 3 4 3 0 0 1 3 3 1 1 4 1 1 2 3 0 1 3 2 1 3 4 1 3 4 1 1 0 2 1
 2 0 4 1 4 3 1 2 3 1 2 2 4 1 3 1 2 4 4 1 0 4 2 3 1 2 2 4 0 0 3 3 2 3 1 3 2
 3 4 1 1 4 2 2 0 4 4 1 2 4 2 3 1 0 0 1 0 4 4 3 1 2 0 3 4 2 3 0 3 3 2 1 4 1
 2 1 0 2 4 4 3 4 0 4 3 4 1 3 3 0 4 0 1 0 2 2 2 4 2 1 3 0 1 4 0 0 3 1 3 1 1
 3 0 1 3 1 2 4 0 2 0 3 3 0 3 0 1 1 3 2 0 2 0 0 4 2 4 3 4 4 3 0 4 4 3 2 2 2
 4 2 1 0 4 0 1 1 1 0 0 2 1 1 4 1 4 0 1 1 3 3 3 3 0 1 0 1 1 1 3 0 1 3 3 1 1
 2 1 1 0 2 1 1 4 0 1 2 2 4 0 3 2 1 3 2 3 1 0 3 3 1 0 4 4 0 1 0 3 1 2 1 3 2
 3 3 3 3 1 0 0 2 1 1 3 0 3 0 1 3 2 3 3 2 3 4 2 0 3 4 0 3 3 2 3 3 1 2 1 2 3
 3 3 0 1 4 0 2 0 0 0 0 1 2 1 4 1 0 1 0 3 3 4 0 0 3 0 0 0 1 4 4 3 4 0 1 1 4
 1 1 1 2 3 0 4 4 0 0 2 4 1 0 0 0 3 1 1 2 0 4 3 2 0 0 4 3 3 2 4 0 1 4 2 0 3
 2 3 2 1 0 4 1 3 0 0 0 0 0 2 3 0 2 3 1 0 2 4 2 3 3 2 3 0 3 2 2 0 2 4 2 3 3
 3 2 4 2 2 4 1

2025-06-18 19:50:00.245501: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


<tf.Tensor: shape=(512,), dtype=float32, numpy=
array([1.5413079e-01, 3.3847861e+00, 4.3262758e+00, 5.0737548e-01,
       0.0000000e+00, 8.3881521e-01, 2.0900130e+00, 0.0000000e+00,
       5.8984798e-01, 3.5898361e+00, 2.5504856e+00, 2.9051447e+00,
       0.0000000e+00, 2.4195631e+00, 0.0000000e+00, 5.6291419e-01,
       9.0719914e-01, 3.5755520e+00, 6.4947176e+00, 0.0000000e+00,
       1.3563871e-02, 6.9377136e-01, 0.0000000e+00, 1.9891839e+00,
       0.0000000e+00, 5.1024747e-01, 3.1205997e+00, 2.2753704e-01,
       0.0000000e+00, 0.0000000e+00, 2.3241444e+00, 2.5290866e+00,
       0.0000000e+00, 5.1916342e+00, 0.0000000e+00, 3.5013666e+00,
       8.6507821e-01, 6.2222662e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 2.0196395e+00,
       3.8102489e+00, 0.0000000e+00, 0.0000000e+00, 9.4992471e-01,
       0.0000000e+00, 8.9378715e-01, 0.0000000e+00, 6.5442743e+00,
       0.0000000e+00, 0.0000000e+00, 3.5641098e+00, 2.2138920e+00,
       0.00000

In [14]:
for batch in dataset.take(1):
    b2_7d = tf.cast(tf.reshape(batch.pop('b2_sale_amt_7d'), (-1, 1)), tf.float32)
    total_amt_1h = tf.reshape(batch.pop('total_pay_amount1'), (-1, 1))
    break

In [15]:
b2_7d - total_amt_1h

<tf.Tensor: shape=(512, 1), dtype=float32, numpy=
array([[ 0.0000000e+00],
       [ 2.7800000e+02],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 1.9000000e+01],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [-8.9999962e-01],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 0.0000000e+00],
       [ 1.6600000e+02],


In [13]:
total_amt_1h

<tf.Tensor: shape=(512, 1), dtype=float32, numpy=
array([[   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [  10.9 ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [   0.  ],
       [  17.9 ],
       [   0.  ],
       [   0. 